In [1]:
%pip install argostranslate


  Using cached argostranslate-1.8.0-py3-none-any.whl (27 kB)
  Using cached ctranslate2-2.24.0-cp310-cp310-win_amd64.whl (14.3 MB)
  Using cached sentencepiece-0.1.96-cp310-cp310-win_amd64.whl (1.1 MB)
  Using cached stanza-1.1.1-py3-none-any.whl (227 kB)
  Using cached PyYAML-6.0-cp310-cp310-win_amd64.whl (151 kB)
  Using cached torch-2.0.1-cp310-cp310-win_amd64.whl (172.3 MB)
                                              0.0/77.1 kB ? eta -:--:--
     ---------------------------------------- 77.1/77.1 kB 2.1 MB/s eta 0:00:00
                                              0.0/5.7 MB ? eta -:--:--
                                              0.1/5.7 MB 1.7 MB/s eta 0:00:04
     --                                       0.3/5.7 MB 4.2 MB/s eta 0:00:02
     ----                                     0.6/5.7 MB 4.1 MB/s eta 0:00:02
     -----                                    0.8/5.7 MB 4.5 MB/s eta 0:00:02
     -------                                  1.0/5.7 MB 4.3 MB/s eta 0:00:02
     -

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import csv
import pandas as pd
from tqdm import tqdm

filename = "data\preprocessed_tweets_tr.csv"

content_list = []

with open(filename, 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    header = next(reader)
    
    for row in reader:
        date = row[0]
        content = row[1]
        hashtags = row[2]
        like_count = float(row[3])
        rt_count = float(row[4])
        followers_count = float(row[5])
        isVerified = bool(row[6])
        language = row[7]
        coordinates = row[8]
        place = row[9]
        source = row[10]
        
        content_list.append(content)

print(len(content_list))

140532


In [2]:
import argostranslate.package
import argostranslate.translate

from_code = "tr"
to_code = "en"

# Download and install Argos Translate package
argostranslate.package.update_package_index()
available_packages = argostranslate.package.get_available_packages()
available_package = list(
    filter(
        lambda x: x.from_code == from_code and x.to_code == to_code, available_packages
    )
)[0]
download_path = available_package.download()
argostranslate.package.install_from_path(download_path)

# Translate
installed_languages = argostranslate.translate.get_installed_languages()
from_lang = list(filter(
        lambda x: x.code == from_code,
        installed_languages))[0]
to_lang = list(filter(
        lambda x: x.code == to_code,
        installed_languages))[0]
translation = from_lang.get_translation(to_lang)
# translatedText = translation.translate(content_list[1])
# print(content_list[1])
# print(translatedText)

In [8]:
# translatedText = translation.translate(content_list[10])
# print(translatedText)
import concurrent.futures
import pandas as pd 

sentences = []
translatedTexts = []
total_sentences = len(content_list)
for i in range(0, total_sentences):
    if(i%100 == 0):
        print("Translated {}/{}".format(i, total_sentences))

    sentence = content_list[i]  # Example: translating sentences 10 to 19
    translatedText = translation.translate(content_list[i])
    sentences.append(sentence)
    translatedTexts.append(translatedText)

#transform both to dataframe with text, translation columns
frame = pd.DataFrame({'text': sentences, 'translation': translatedTexts})
# save frame to csv
frame.to_csv('data/translated_tweets_ar.csv', index=False)



Translated 0/140532
Translated 100/140532


In [3]:
import concurrent.futures

sentences = content_list[10:20]  # Example: translating sentences 10 to 19

def translate_sentence(index, sentence):
    return index, translation.translate(sentence)

# Create a ThreadPoolExecutor with a maximum of 5 worker threads
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Submit each sentence for translation asynchronously
    translation_futures = [executor.submit(translate_sentence, i, sentence) for i, sentence in enumerate(sentences)]

    # Retrieve the translated sentences as they become available
    translations = [None] * len(sentences)
    for future in concurrent.futures.as_completed(translation_futures):
        index, translated_sentence = future.result()
        translations[index] = translated_sentence

print(translations)

In [ ]:
# import csv

# filename = "/Users/saraclaus/Desktop/preprocessed_tweets_ar.csv"

# content_list = []

# with open(filename, 'r', encoding='utf-8') as file:
#     reader = csv.reader(file)
#     header = next(reader)
    
#     for row in reader:
#         date = row[0]
#         content = row[1]
#         hashtags = row[2]
#         like_count = float(row[3])
#         rt_count = float(row[4])
#         followers_count = float(row[5])
#         isVerified = bool(row[6])
#         language = row[7]
#         coordinates = row[8]
#         place = row[9]
#         source = row[10]
        
#         content_list.append(content)

# import argostranslate.package
# import argostranslate.translate

# from_code = "ar"
# to_code = "en"

# # Download and install Argos Translate package
# argostranslate.package.update_package_index()
# available_packages = argostranslate.package.get_available_packages()
# available_package = list(
#     filter(
#         lambda x: x.from_code == from_code and x.to_code == to_code, available_packages
#     )
# )[0]
# download_path = available_package.download()
# argostranslate.package.install_from_path(download_path)

# # Translate
# installed_languages = argostranslate.translate.get_installed_languages()
# from_lang = list(filter(lambda x: x.code == from_code, installed_languages))[0]
# to_lang = list(filter(lambda x: x.code == to_code, installed_languages))[0]
# translation = from_lang.get_translation(to_lang)
# i = 0
# translated_list = []
# for content in content_list:
#     i += 1
#     translated_text = translation.translate(content)
#     translated_list.append(translated_text)
#     print(i)
